<a href="https://colab.research.google.com/github/ivanvovk/compressed-tts/blob/master/tests/conv_distilling/conv_svd_distilling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Distilling convolutions in Tacotron 2

Firstly, download Tacotron 2 checkpoint: https://drive.google.com/file/d/1c5ZTuT7J08wLUoVZ2KkUs_VdZuJ86ZqA/view.

Then install requirements from the project root folder: `pip install -r requirements.txt`

In [0]:
from google.colab import drive
import os 


drive.mount('/content/drive')

import sys
sys.path.append('drive/My Drive/Colab Notebooks/Distillation/TTS/compressed-tts-master/tests/conv_distilling')
os.chdir('drive/My Drive/Colab Notebooks/Distillation/TTS/compressed-tts-master/tests/conv_distilling')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!pip install unidecode

     |████████████████████████████████| 245kB 6.7MB/s 


In [0]:
import warnings
import torch.optim as optim

warnings.filterwarnings('ignore')

import sys
sys.path.insert(0, "../../")
sys.path.insert(0, "../../tacotron2/")

import json
import torch
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from IPython.display import Audio
from IPython.display import display

from audio.vocoders import griffin_lim

import re
from tacotron2.text import cleaners
from tacotron2.text.symbols import symbols

import sys
from torch import nn
import torch
import torch.nn.functional as F
from tqdm import tqdm_notebook as tqdm

from math import sqrt
from tacotron2.layers import ConvNorm

In [0]:
# Mappings from symbol to numeric ID and vice versa:
_symbol_to_id = {s: i for i, s in enumerate(symbols)}
_id_to_symbol = {i: s for i, s in enumerate(symbols)}

# Regular expression matching text enclosed in curly braces:
_curly_re = re.compile(r'(.*?)\{(.+?)\}(.*)')


def text_to_sequence(text, cleaner_names):
  '''Converts a string of text to a sequence of IDs corresponding to the symbols in the text.

    The text can optionally have ARPAbet sequences enclosed in curly braces embedded
    in it. For example, "Turn left on {HH AW1 S S T AH0 N} Street."

    Args:
      text: string to convert to a sequence
      cleaner_names: names of the cleaner functions to run the text through

    Returns:
      List of integers corresponding to the symbols in the text
  '''
  sequence = []

  # Check for curly braces and treat their contents as ARPAbet:
  while len(text):
    m = _curly_re.match(text)
    if not m:
      sequence += _symbols_to_sequence(_clean_text(text, cleaner_names))
      break
    sequence += _symbols_to_sequence(_clean_text(m.group(1), cleaner_names))
    sequence += _arpabet_to_sequence(m.group(2))
    text = m.group(3)

  return sequence


def sequence_to_text(sequence):
  '''Converts a sequence of IDs back to a string'''
  result = ''
  for symbol_id in sequence:
    if symbol_id in _id_to_symbol:
      s = _id_to_symbol[symbol_id]
      # Enclose ARPAbet back in curly braces:
      if len(s) > 1 and s[0] == '@':
        s = '{%s}' % s[1:]
      result += s
  return result.replace('}{', ' ')


def _clean_text(text, cleaner_names):
  for name in cleaner_names:
    cleaner = getattr(cleaners, name)
    if not cleaner:
      raise Exception('Unknown cleaner: %s' % name)
    text = cleaner(text)
  return text


def _symbols_to_sequence(symbols):
  return [_symbol_to_id[s] for s in symbols if _should_keep_symbol(s)]


def _arpabet_to_sequence(text):
  return _symbols_to_sequence(['@' + s for s in text.split()])


def _should_keep_symbol(s):
  return s in _symbol_to_id and s is not '_' and s is not '~'

In [0]:
class Encoder(nn.Module):
    """Encoder module:
        - Three 1-d convolution banks
        - Bidirectional LSTM
    """
    def __init__(self, config):
        super(Encoder, self).__init__()

        convolutions = []
        for _ in range(config["encoder_n_convolutions"]):
            conv_layer = nn.Sequential(
                ConvNorm(config["encoder_embedding_dim"],
                         config["encoder_embedding_dim"],
                         kernel_size=config["encoder_kernel_size"], stride=1,
                         padding=int((config["encoder_kernel_size"] - 1) / 2),
                         dilation=1, w_init_gain='relu'),
                nn.BatchNorm1d(config["encoder_embedding_dim"]))
            convolutions.append(conv_layer)
        self.convolutions = nn.ModuleList(convolutions)

        self.lstm = nn.LSTM(config["encoder_embedding_dim"],
                            int(config["encoder_embedding_dim"] / 2), 1,
                            batch_first=True, bidirectional=True)

    def forward(self, x, input_lengths):
        for conv in self.convolutions:
            x = F.dropout(F.relu(conv(x)), 0.5, self.training)

        x = x.transpose(1, 2)

        # pytorch tensor are not reversible, hence the conversion
        input_lengths = input_lengths.cpu().numpy()
        x = nn.utils.rnn.pack_padded_sequence(
            x, input_lengths, batch_first=True)

        self.lstm.flatten_parameters()
        outputs, _ = self.lstm(x)

        outputs, _ = nn.utils.rnn.pad_packed_sequence(
            outputs, batch_first=True)

        return outputs

    def inference(self, x):
        for conv in self.convolutions:
            x = F.dropout(F.relu(conv(x)), 0.5, self.training)

        x = x.transpose(1, 2)

        self.lstm.flatten_parameters()
        outputs, _ = self.lstm(x)

        return outputs

class ConvModule(torch.nn.Module):
    def __init__(self, config):
        super(ConvModule, self).__init__()
        
        self.embedding = torch.nn.Embedding(
            config["n_symbols"],
            config["symbols_embedding_dim"]
        )
        std = sqrt(2.0 / (config["n_symbols"] + config["symbols_embedding_dim"]))
        val = sqrt(3.0) * std  # uniform bounds for std
        self.embedding.weight.data.uniform_(-val, val)
        
        convolutions = []
        for _ in range(config["encoder_n_convolutions"]):
            conv_layer = torch.nn.Sequential(
                ConvNorm(config["encoder_embedding_dim"],
                         config["encoder_embedding_dim"],
                         kernel_size=config["encoder_kernel_size"], stride=1,
                         padding=int((config["encoder_kernel_size"] - 1) / 2),
                         dilation=1, w_init_gain='relu'),
                torch.nn.BatchNorm1d(config["encoder_embedding_dim"]))
            convolutions.append(conv_layer)
        self.convolutions = torch.nn.ModuleList(convolutions)
        
    def forward(self, x):
        outputs = self.embedding(x).transpose(1, 2)

        self.intermediate_activations = []

        for conv in self.convolutions:
            outputs = F.dropout(F.relu(conv(outputs)), 0.5, self.training)
            self.intermediate_activations.append(outputs)
        return outputs
    
    def inference(self, x):
        outputs = self.embedding(x).transpose(1, 2)
        for conv in self.convolutions:
            outputs = F.dropout(F.relu(conv(outputs)), 0.5, self.training)
        return outputs

## **1 DEVICE**

In [0]:
DEVICE = 'cuda:0'

___
## **1 TEACHER**

In [0]:
TACOTRON_CONFIG=json.load(open('./../../tacotron2/config.json', 'r'))
TACOTRON_CHECKPT='./../../checkpoints/tacotron2_statedict.pt'

teacher = ConvModule(TACOTRON_CONFIG)
teacher.load_state_dict(torch.load('TeacherParameters.torch'))
teacher.to(DEVICE);

## **2 STUDENT**


In [0]:
class Student(torch.nn.Module):
    def __init__(self, channels=256):
        super(Student, self).__init__()
        
        self.embedding = torch.nn.Embedding(148, 512)
        
        std = sqrt(2.0 / (148 + channels))
        val = sqrt(3.0) * std  # uniform bounds for std
        self.embedding.weight.data.uniform_(-val, val)
        
        convolutions = [torch.nn.Sequential(
            ConvNorm(512,
                        channels,
                        kernel_size=5, stride=1,
                        padding=int((5 - 1) / 2),
                        dilation=1, w_init_gain='relu'),
            torch.nn.BatchNorm1d(channels)),

                        torch.nn.Sequential(
            ConvNorm(channels,
                        channels,
                        kernel_size=5, stride=1,
                        padding=int((5 - 1) / 2),
                        dilation=1, w_init_gain='relu'),
                        torch.nn.BatchNorm1d(channels)),

                        torch.nn.Sequential(
                        ConvNorm(channels,
                                    512,
                                    kernel_size=5, stride=1,
                                    padding=int((5 - 1) / 2),
                                    dilation=1, w_init_gain='relu'),
                        torch.nn.BatchNorm1d(512))]
        
            
        self.convolutions = torch.nn.ModuleList(convolutions)
        
    def forward(self, x):
        outputs = self.embedding(x).transpose(1, 2)

        self.intermediate_activations = []

        for conv in self.convolutions:
            outputs = F.dropout(F.relu(conv(outputs)), 0.5, self.training)
            self.intermediate_activations.append(outputs)

        return outputs
    
    def inference(self, x):
        outputs = self.embedding(x).transpose(1, 2)
        for conv in self.convolutions:
            outputs = F.dropout(F.relu(conv(outputs)), 0.5, self.training)
        return outputs
    
    def nparams(self):
        return sum(p.numel() for p in self.parameters() if p.requires_grad)

___
## **3 Dataset**

In [0]:
import random

import torch
import torch.utils.data

from utils import load_filepaths_and_text

class TextDataset(torch.utils.data.Dataset):
    def __init__(
            self,
            audiopaths_and_texts,
            config,
            shuffle=True,
            teacher=None,
            device='cuda:0'
        ):
        self.audiopaths_and_texts = load_filepaths_and_text(
            audiopaths_and_texts, config['sort_by_length'])
        self.text_cleaners = config['text_cleaners']
        random.seed(1234)
        if shuffle:
            random.shuffle(self.audiopaths_and_texts)

        self.gt_module = teacher
        _ = self.gt_module.to(device).eval()
        
        self.device = device

    def get_text(self, text):
        text_norm = torch.LongTensor(text_to_sequence(text, [self.text_cleaners]))
        return text_norm

    def __getitem__(self, index):
        audiopath_and_text = self.audiopaths_and_texts[index]
        _, text = audiopath_and_text[0], audiopath_and_text[1]
        
        text = self.get_text(text)
        with torch.no_grad():
            return (text.to(self.device), self.gt_module(text[None].to(self.device)).squeeze(0))

    def __len__(self):
        return len(self.audiopaths_and_texts)
    
    
class TextCollate(object):
    def __init__(self):
        pass

    def __call__(self, batch):
        # Right zero-pad all one-hot text sequences to max input length
        max_input_len = max(map(len, [sample[0] for sample in batch]))

        # PAD TEXT
        text_padded = torch.zeros(len(batch), max_input_len)
        for i, text in enumerate([sample[0] for sample in batch]):
            text_padded[i, :text.size(0)] = text
        
        # PAD CONVOLUTIONS OUTPUTS
        targets_padded = torch.zeros(len(batch), 512, max_input_len)
        for i, target in enumerate([sample[1] for sample in batch]):
            targets_padded[i, :target.size(0), :target.size(1)] = target
            
        return (text_padded, targets_padded)

In [0]:
from torch.utils.data import DataLoader

N_EPOCHS=100
BACTH_SIZE=16
ITER_PER_VERBOSE_TRAIN=5
ITER_PER_VERBOSE_TEST=50

In [0]:
train_dataset = TextDataset('./filelists/ljs_audio_text_train_filelist.txt', TACOTRON_CONFIG, teacher=teacher, device=DEVICE)
test_dataset = TextDataset('./filelists/ljs_audio_text_test_filelist.txt', TACOTRON_CONFIG, teacher=teacher, device=DEVICE)

train_loader = DataLoader(
    dataset=train_dataset,
    batch_size=BACTH_SIZE,
    collate_fn=TextCollate(),
    drop_last=True
)
test_loader = DataLoader(
    dataset=test_dataset,
    batch_size=BACTH_SIZE,
    collate_fn=TextCollate(),
    drop_last=True
)

In [0]:
class Diag:
    def __init__(self):
        pass
    def __call__(self, x):
        oup, k_size = x.shape

        diagonal = []
        for index in range(oup):
            diagonal.append(torch.diag(x[index]).reshape(1, k_size, k_size))

        return torch.cat(diagonal)

class Decomposition:
    def __init__(self, conv):
        self.conv = conv
        self.diagonal = Diag()

    def low_rank(self, rank):
        u, s, v = torch.svd(self.conv)
        S = self.diagonal(s)
        S = S[:, :rank, :rank]

        v = torch.transpose(v, 1, 2)
        u, v = u[:, :, :rank], v[:, :rank, :]
        us = torch.matmul(u, S)
        return us, v

    def substitute_conv(self, rank, random_init=True):
        us, v = self.low_rank(rank)
        oup_us, inp_us, h_us, w_us = us.shape
        oup_v, inp_v, h_v, w_v = v.shape

        us_conv = nn.Conv2d(inp_us, oup_us, kernel_size=(h_us, w_us), padding=(h_us//2, w_us//2), bias=False)
        v_conv = nn.Conv2d(inp_v, oup_v, kernel_size=(h_v, w_v), padding=(h_v//2, w_v//2), bias=False)

        if not random_init:
            us_conv.weight = nn.Parameter(us)
            v_conv.weight = nn.Parameter(v)

        self.us_conv = us_conv
        self.v_conv = v_conv
        return us_conv, v_conv   


class DistillBlock:
    def __init__(self, student, device = 'cuda:0'):
        self.device = device

        self.student = student.to(device)

        self.lr = 1e-3
        self.optimizer = optim.Adam(self.student.parameters(), lr=self.lr)
        self.MSE = nn.MSELoss().to(device)

        self.batches_passed = 0
        self.loss_tracker = []

        self.training_stats = [[] for _ in range(100)]
    def l2(self, x1, x2):
        return torch.norm(x1 - x2)

    def low_rank_target(self, batch, rank):
        us, v = Decomposition(batch).low_rank(rank)
        return torch.matmul(us, v)

    def transfer(self, batch, rank, epoch):
        """
        Activation Tensor ~ data
        """
        self.optimizer.zero_grad()

        data, teacher_prediction = batch
        data, teacher_prediction = data.to(self.device), teacher_prediction.to(self.device)

        if rank != 126:
            teacher_prediction = self.low_rank_target(teacher_prediction, rank)

        student_prediction = self.student(data.long().to(self.device))

        distance = self.l2(teacher_prediction, student_prediction)
        reconstruction_loss = self.MSE(teacher_prediction, student_prediction)

        (0.5*distance + 0.5*reconstruction_loss).backward()
        self.optimizer.step()

        if (self.batches_passed + 1)%90 == 0:
            print(f'Batch {self.batches_passed + 1} \nMSE Loss:', reconstruction_loss.item(), '\nDistance loss:', distance.item())

            self.training_stats[epoch].append([reconstruction_loss.item(), distance.item()])

        self.batches_passed += 1
        self.loss_tracker.append([reconstruction_loss.item(), distance.item()])

# EXPERIMENT SIMPLE DISTILL



In [0]:
batch = next(iter(train_loader))

In [0]:
batch[0].shape

torch.Size([16, 126])

In [0]:
%%timeit
teacher(batch[0].long().cpu())

1 loop, best of 3: 10.6 s per loop


In [0]:
%%timeit
student(batch[0].long().cpu()) # TEACHER

1 loop, best of 3: 229 ms per loop


In [0]:
%%timeit
student(batch[0].long().cpu()) #STUDENT256

10 loops, best of 3: 119 ms per loop


In [0]:
%%timeit
student(batch[0].long().cpu()) #STUDENT128

10 loops, best of 3: 60.9 ms per loop


In [0]:
%%timeit
student(batch[0].long().cpu()) #STUDENT64

10 loops, best of 3: 38.7 ms per loop


In [0]:
for channels in [64, 128, 256]:
    student = Student(channels=channels)
    student.to(DEVICE);

    distillation_process_simple = DistillBlock(student)

    test_loss = []

    for epoch, rank in enumerate(range(40)):
        for index, batch in tqdm(enumerate(train_loader), total=len(train_loader)):

            distillation_process_simple.transfer(batch, rank=126, epoch=epoch)

        if epoch == 30:
            for param_group in distillation_process_simple.optimizer.param_groups:
                param_group['lr'] = (1e-3)/2

        current_student = distillation_process_simple.student

        loss = []
        mse = []
        MSE = nn.MSELoss()

        for index, batch in tqdm(enumerate(test_loader), total=len(test_loader)):
            data, label = batch
            
            pred_student = current_student(data.long().to(DEVICE))
            loss.append(distillation_process_simple.l2(pred_student, label.to(DEVICE)).item())
            mse.append(MSE(pred_student, label.to(DEVICE)).item())

        print('TEST LOSS:', np.mean(loss), np.mean(mse))
        
        a,b = np.array(distillation_process_simple.training_stats[epoch]).mean(axis=0)

        test_loss.append([a, b, np.mean(loss), np.mean(mse)])

    test_loss = np.array(test_loss)
    pd.DataFrame(test_loss).to_csv(f'Loss{channels}.csv', index=False)

Batch 90 
MSE Loss: 0.24773044884204865 
Distance loss: 531.1192626953125
Batch 180 
MSE Loss: 0.188603475689888 
Distance loss: 481.41033935546875
Batch 270 
MSE Loss: 0.16689904034137726 
Distance loss: 440.62188720703125
Batch 360 
MSE Loss: 0.17660370469093323 
Distance loss: 459.5907897949219
Batch 450 
MSE Loss: 0.14068962633609772 
Distance loss: 415.7877502441406
Batch 540 
MSE Loss: 0.14750586450099945 
Distance loss: 422.8929748535156
Batch 630 
MSE Loss: 0.1394909769296646 
Distance loss: 431.5805358886719
Batch 720 
MSE Loss: 0.130418062210083 
Distance loss: 412.15673828125


TEST LOSS: 417.62625515845514 0.13971256128241938


Batch 810 
MSE Loss: 0.1273440718650818 
Distance loss: 392.92999267578125
Batch 900 
MSE Loss: 0.14979428052902222 
Distance loss: 438.9271240234375
Batch 990 
MSE Loss: 0.1260814219713211 
Distance loss: 362.1782531738281
Batch 1080 
MSE Loss: 0.13238085806369781 
Distance loss: 423.0089416503906
Batch 1170 
MSE Loss: 0.11887858062982559 
Distance loss: 402.0691833496094
Batch 1260 
MSE Loss: 0.12081708759069443 
Distance loss: 392.9356994628906
Batch 1350 
MSE Loss: 0.13143117725849152 
Distance loss: 412.4512939453125
Batch 1440 
MSE Loss: 0.1357337385416031 
Distance loss: 413.808349609375
Batch 1530 
MSE Loss: 0.12051022797822952 
Distance loss: 394.9439697265625


TEST LOSS: 393.1205365580897 0.12380567817918715


Batch 1620 
MSE Loss: 0.11860861629247665 
Distance loss: 394.28759765625
Batch 1710 
MSE Loss: 0.1271081417798996 
Distance loss: 397.8354187011719
Batch 1800 
MSE Loss: 0.1364111751317978 
Distance loss: 414.8397216796875
Batch 1890 
MSE Loss: 0.13640306890010834 
Distance loss: 399.7376403808594
Batch 1980 
MSE Loss: 0.12223826348781586 
Distance loss: 377.0880432128906
Batch 2070 
MSE Loss: 0.111941859126091 
Distance loss: 386.6206359863281
Batch 2160 
MSE Loss: 0.13026316463947296 
Distance loss: 408.0074157714844
Batch 2250 
MSE Loss: 0.11782119423151016 
Distance loss: 386.7875671386719
Batch 2340 
MSE Loss: 0.1176612600684166 
Distance loss: 380.2395935058594


...

TEST LOSS: 344.84081342143395 0.0952641855805151


Batch 29700 
MSE Loss: 0.09290479868650436 
Distance loss: 356.5104675292969
Batch 29790 
MSE Loss: 0.10238903015851974 
Distance loss: 343.8992919921875
Batch 29880 
MSE Loss: 0.09821634739637375 
Distance loss: 372.0069580078125
Batch 29970 
MSE Loss: 0.10948467254638672 
Distance loss: 344.0793762207031
Batch 30060 
MSE Loss: 0.11132437735795975 
Distance loss: 373.5389099121094
Batch 30150 
MSE Loss: 0.0900331661105156 
Distance loss: 353.0527648925781
Batch 30240 
MSE Loss: 0.09491150826215744 
Distance loss: 358.1759033203125
Batch 30330 
MSE Loss: 0.10404183715581894 
Distance loss: 358.746337890625
Batch 30420 
MSE Loss: 0.09679563343524933 
Distance loss: 355.0760498046875


TEST LOSS: 344.58690914030996 0.09511144651520637


Batch 30510 
MSE Loss: 0.09115516394376755 
Distance loss: 355.24578857421875
Batch 30600 
MSE Loss: 0.09948430210351944 
Distance loss: 344.9440612792969
Batch 30690 
MSE Loss: 0.1090228334069252 
Distance loss: 376.8359069824219
Batch 30780 
MSE Loss: 0.0913786068558693 
Distance loss: 346.0805969238281
Batch 30870 
MSE Loss: 0.10005924105644226 
Distance loss: 351.8131408691406
Batch 30960 
MSE Loss: 0.0980885922908783 
Distance loss: 351.7746887207031
Batch 31050 
MSE Loss: 0.09749511629343033 
Distance loss: 334.3877868652344
Batch 31140 
MSE Loss: 0.09648223966360092 
Distance loss: 346.6094970703125
Batch 31230 
MSE Loss: 0.08929605036973953 
Distance loss: 352.64324951171875


TEST LOSS: 344.8053933420489 0.09524708169121895


# EXPERIMENT SVD DISTILL


In [0]:
for channels in [64, 128, 256]:

    student = Student(channels=channels)
    student.to(DEVICE);

    distillation_process_svd = DistillBlock(student)
    test_loss_svd = []

    ranks = list(range(2, 126, 6)) + [126] * 19

    for epoch, rank in enumerate(ranks):
        for index, batch in tqdm(enumerate(train_loader), total=len(train_loader)):
            distillation_process_svd.transfer(batch, rank=rank, epoch=epoch)

        current_student = distillation_process_svd.student

        if epoch == 30:
            for param_group in distillation_process_simple.optimizer.param_groups:
                param_group['lr'] = (1e-3)/2

        loss = []
        mse = []
        MSE = nn.MSELoss()

        for index, batch in tqdm(enumerate(test_loader), total=len(test_loader)):
            data, label = batch

            pred_student = current_student(data.long().to(DEVICE))
            loss.append(distillation_process_svd.l2(pred_student, label.to(DEVICE)).item())
            mse.append(MSE(pred_student, label.to(DEVICE)).item())

        a,b = np.array(distillation_process_svd.training_stats[epoch]).mean(axis=0)
        test_loss_svd.append([a,b, np.mean(loss), np.mean(mse)])

    test_loss_svd = np.array(test_loss_svd)
    pd.DataFrame(test_loss_svd).to_csv(f'SVD_Loss{channels}.csv', index=False)

Batch 90 
MSE Loss: 0.08671711385250092 
Distance loss: 314.23516845703125
Batch 180 
MSE Loss: 0.04761989042162895 
Distance loss: 241.89938354492188
Batch 270 
MSE Loss: 0.04192257672548294 
Distance loss: 220.8325653076172
Batch 360 
MSE Loss: 0.04600852355360985 
Distance loss: 234.5797576904297
Batch 450 
MSE Loss: 0.03659573197364807 
Distance loss: 212.05856323242188
Batch 540 
MSE Loss: 0.0403650663793087 
Distance loss: 221.22218322753906
Batch 630 
MSE Loss: 0.03749001771211624 
Distance loss: 223.7415313720703
Batch 720 
MSE Loss: 0.03421683609485626 
Distance loss: 211.1122589111328


Batch 810 
MSE Loss: 0.0890359878540039 
Distance loss: 328.5553894042969
Batch 900 
MSE Loss: 0.09908270835876465 
Distance loss: 356.9798278808594
Batch 990 
MSE Loss: 0.08807192742824554 
Distance loss: 302.7021789550781
Batch 1080 
MSE Loss: 0.08811410516500473 
Distance loss: 345.1116943359375
Batch 1170 
MSE Loss: 0.07913124561309814 
Distance loss: 328.03717041015625
Batch 1260 
MSE Loss: 0.08009447157382965 
Distance loss: 319.9326477050781
Batch 1350 
MSE Loss: 0.08703190833330154 
Distance loss: 335.6315612792969
Batch 1440 
MSE Loss: 0.08863729238510132 
Distance loss: 334.39794921875
Batch 1530 
MSE Loss: 0.07808465510606766 
Distance loss: 317.9115905761719


Batch 1620 
MSE Loss: 0.09794386476278305 
Distance loss: 358.29730224609375
Batch 1710 
MSE Loss: 0.10344120860099792 
Distance loss: 358.8918151855469
Batch 1800 
MSE Loss: 0.10858985036611557 
Distance loss: 370.1263427734375
Batch 1890 
MSE Loss: 0.10990608483552933 
Distance loss: 358.8177185058594
Batch 1980 
MSE Loss: 0.1013718843460083 
Distance loss: 343.3981018066406
Batch 2070 
MSE Loss: 0.09107054024934769 
Distance loss: 348.720703125
Batch 2160 
MSE Loss: 0.10304620116949081 
Distance loss: 362.8885192871094
Batch 2250 
MSE Loss: 0.09417816251516342 
Distance loss: 345.8087158203125
Batch 2340 
MSE Loss: 0.0937432199716568 
Distance loss: 339.39898681640625


Batch 2430 
MSE Loss: 0.10386402159929276 
Distance loss: 359.62457275390625
Batch 2520 
MSE Loss: 0.10571540892124176 
Distance loss: 364.007080078125
Batch 2610 
MSE Loss: 0.08314486593008041 
Distance loss: 328.0508728027344
Batch 2700 
MSE Loss: 0.10863757133483887 
Distance loss: 364.1481628417969
Batch 2790 
MSE Loss: 0.10022810846567154 
Distance loss: 364.7096862792969
Batch 2880 
MSE Loss: 0.10693022608757019 
Distance loss: 367.2870178222656
Batch 2970 
MSE Loss: 0.10122043639421463 
Distance loss: 372.1236877441406
Batch 3060 
MSE Loss: 0.09791991859674454 
Distance loss: 351.47210693359375


Batch 3150 
MSE Loss: 0.1177615225315094 
Distance loss: 374.0078125
Batch 3240 
MSE Loss: 0.11268182843923569 
Distance loss: 390.26885986328125
Batch 3330 
MSE Loss: 0.09899051487445831 
Distance loss: 353.3882751464844
Batch 3420 
MSE Loss: 0.10257162898778915 
Distance loss: 350.2555847167969
Batch 3510 
MSE Loss: 0.1065278947353363 
Distance loss: 355.721923828125
Batch 3600 
MSE Loss: 0.1039942130446434 
Distance loss: 385.0119323730469
Batch 3690 
MSE Loss: 0.10290868580341339 
Distance loss: 359.14239501953125
Batch 3780 
MSE Loss: 0.10846485942602158 
Distance loss: 374.6864929199219
Batch 3870 
MSE Loss: 0.11080418527126312 
Distance loss: 356.4815673828125


...

Batch 10170 
MSE Loss: 0.1152285560965538 
Distance loss: 384.9682312011719
Batch 10260 
MSE Loss: 0.09305427223443985 
Distance loss: 345.94970703125
Batch 10350 
MSE Loss: 0.1204797625541687 
Distance loss: 379.6019592285156
Batch 10440 
MSE Loss: 0.1126708984375 
Distance loss: 351.68450927734375
Batch 10530 
MSE Loss: 0.10390389710664749 
Distance loss: 362.0522155761719
Batch 10620 
MSE Loss: 0.08814125508069992 
Distance loss: 354.4534606933594
Batch 10710 
MSE Loss: 0.10273230820894241 
Distance loss: 368.0963134765625
Batch 10800 
MSE Loss: 0.10327636450529099 
Distance loss: 352.6582946777344
Batch 10890 
MSE Loss: 0.0921047031879425 
Distance loss: 352.8400573730469


Batch 10980 
MSE Loss: 0.10960045456886292 
Distance loss: 360.81549072265625
Batch 11070 
MSE Loss: 0.09314091503620148 
Distance loss: 337.1773681640625
Batch 11160 
MSE Loss: 0.10568294674158096 
Distance loss: 363.951171875
Batch 11250 
MSE Loss: 0.08855152130126953 
Distance loss: 339.61865234375
Batch 11340 
MSE Loss: 0.11479588598012924 
Distance loss: 344.2257995605469
Batch 11430 
MSE Loss: 0.10921986401081085 
Distance loss: 363.8954772949219
Batch 11520 
MSE Loss: 0.09862810373306274 
Distance loss: 344.6307373046875
Batch 11610 
MSE Loss: 0.0997072383761406 
Distance loss: 325.85919189453125
Batch 11700 
MSE Loss: 0.10643300414085388 
Distance loss: 367.6199035644531


Batch 11790 
MSE Loss: 0.10885931551456451 
Distance loss: 376.55322265625
Batch 11880 
MSE Loss: 0.0989178791642189 
Distance loss: 358.947509765625
Batch 11970 
MSE Loss: 0.10415579378604889 
Distance loss: 367.1683654785156
Batch 12060 
MSE Loss: 0.09532532840967178 
Distance loss: 355.67767333984375
Batch 12150 
MSE Loss: 0.09406483173370361 
Distance loss: 357.6536560058594
Batch 12240 
MSE Loss: 0.10741643607616425 
Distance loss: 379.8851318359375
Batch 12330 
MSE Loss: 0.09916345775127411 
Distance loss: 346.7381896972656
Batch 12420 
MSE Loss: 0.0913807824254036 
Distance loss: 341.73126220703125


Batch 12510 
MSE Loss: 0.10933389514684677 
Distance loss: 365.3133544921875
Batch 12600 
MSE Loss: 0.08770351856946945 
Distance loss: 361.6089782714844
Batch 12690 
MSE Loss: 0.10679361969232559 
Distance loss: 362.2540283203125
Batch 12780 
MSE Loss: 0.11252035945653915 
Distance loss: 354.0625305175781
Batch 12870 
MSE Loss: 0.10399411618709564 
Distance loss: 366.8832702636719
Batch 12960 
MSE Loss: 0.09780935198068619 
Distance loss: 355.80633544921875
Batch 13050 
MSE Loss: 0.1072172001004219 
Distance loss: 361.75982666015625
Batch 13140 
MSE Loss: 0.0983680784702301 
Distance loss: 357.9485778808594
Batch 13230 
MSE Loss: 0.10236117243766785 
Distance loss: 360.5192565917969


Buffered data was truncated after reaching the output size limit.